In [1]:
from scipy.stats import t, f
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import wooldridge

ここで，

は有意水準である。

として信頼区間の解釈を考えよう。

（間違った解釈）
母集団の（真の）の値が信頼区間に入る確率は95％である。

（正しい解釈）
標本を取りOLS推定値を計算するというプロセスを100回繰り返すとしよう。それぞれの標本は異なるため推定値
と値も異なることになり，100の異なる信頼区間が生成される。その内，母集団の（真の）の値が信頼区間に入るのは概ね95回であり，概ね5回は入らない。更に，100回ではなく無限回繰り返すと，計算した信頼区間の95%には真の値が入り，5%には入らない。

紛らわしい解釈だが，次の理由でこの様な解釈となる。まずサイコロを考えよう。6が出る確率は1/6だが，これはサイコロの目がランダム変数だからである。当たり前だが，ランダム変数の場合に確率を考えることができる。このことを踏まえて，母集団の（真の）の値を考えよう。サイコロの目と違って，は定数として与えられており，ランダム変数ではない。従って，１つの信頼区間を計算した後に，その区間に（ランダム変数ではない）真の値が入るかについての確率を考えることは無意味なのである。更に言うと，が信頼区間に入っているのか，もしくは，入っていないかの２つのケースしかなく，確率は1もしくは0にしかならないし，0と1の間の確率はあり得ないという事になる。一方，信頼区間の上限と下限の値はランダムであり，異なる標本を使い複数回にわたって信頼区間を計算すると，異なる上限値・下限値となる。この場合，ランダムな信頼区間が（観測不可能だが）定数を含むかの確率を考えることは意味があることであり，それが信頼区間の解釈となる。

In [2]:
a = 0.001
dof = 30-5-1

t_right = t.ppf(1-a, dof)  # t.ppfについてはscipy.statsを参照
t_right

t_both = t.ppf(1-a/2, dof)
t_both = abs(t.ppf(a/2,dof))


#p-value
1-t.cdf(t_right, dof)   # t.cdfについてはscipy.statsを参照

0.0010000000002238218

In [3]:
gpa = wooldridge.data('gpa1')
wooldridge.data('gpa1',description=True)

name of dataset: gpa1
no of variables: 29
no of observations: 141

+----------+--------------------------------+
| variable | label                          |
+----------+--------------------------------+
| age      | in years                       |
| soph     | =1 if sophomore                |
| junior   | =1 if junior                   |
| senior   | =1 if senior                   |
| senior5  | =1 if fifth year senior        |
| male     | =1 if male                     |
| campus   | =1 if live on campus           |
| business | =1 if business major           |
| engineer | =1 if engineering major        |
| colGPA   | MSU GPA                        |
| hsGPA    | high school GPA                |
| ACT      | 'achievement' score            |
| job19    | =1 if job <= 19 hours          |
| job20    | =1 if job >= 20 hours          |
| drive    | =1 if drive to campus          |
| bike     | =1 if bicycle to campus        |
| walk     | =1 if walk to campus           |
| voluntr  | 

In [4]:
formula_gpa = 'colGPA ~ hsGPA + ACT + skipped'
res_gpa = ols(formula_gpa, data=gpa).fit()

print(res_gpa.summary(slim = True))

                            OLS Regression Results                            
Dep. Variable:                 colGPA   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.217
No. Observations:                 141   F-statistic:                     13.92
Covariance Type:            nonrobust   Prob (F-statistic):           5.65e-08
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3896      0.332      4.191      0.000       0.734       2.045
hsGPA          0.4118      0.094      4.396      0.000       0.227       0.597
ACT            0.0147      0.011      1.393      0.166      -0.006       0.036
skipped       -0.0831      0.026     -3.197      0.002      -0.135      -0.032

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [12]:
def significance_check(res, a=0.05):
    """
    定数項がある場合のt検定をおこなう。
    
    引数：
        res = statsmodelsによるOLS推定結果
        a = 有意水準（デフォルト：0.05）
    返り値：
        帰無仮説を棄却する場合はTrueを返す。
    """
    
    dof = res.nobs-res.df_model-1
    result = abs(res.tvalues) > t.ppf(1-a/2, dof)
    
    return pd.Series(result, index=res.tvalues.index)

In [13]:
significance_check(res_gpa, 0.05)

Intercept     True
hsGPA         True
ACT          False
skipped       True
dtype: bool